In [1]:
from nordpool import elspot
import pandas as pd
import numpy as np
import openmeteo_requests
import requests_cache



### Custom functions

In [2]:
## Add .. to path to allow importing from parent directory
import sys
sys.path.append("..")

from fun.api_f import fetch_nordpool_history


ImportError: cannot import name 'fetch_nordpool_history' from 'fun.api_f' (c:\GIT\01. Research\el-price-prediction\src\notebooks\..\fun\api_f.py)

In [ ]:
df = pd.DataFrame(fetch_nordpool_data(area="EE")['values'])
df

Prices for EE: {'values': [{'start': datetime.datetime(2025, 1, 9, 23, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 0, 0, tzinfo=tzutc()), 'value': 79.76}, {'start': datetime.datetime(2025, 1, 10, 0, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 1, 0, tzinfo=tzutc()), 'value': 79.74}, {'start': datetime.datetime(2025, 1, 10, 1, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 2, 0, tzinfo=tzutc()), 'value': 54.95}, {'start': datetime.datetime(2025, 1, 10, 2, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 3, 0, tzinfo=tzutc()), 'value': 50.94}, {'start': datetime.datetime(2025, 1, 10, 3, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 4, 0, tzinfo=tzutc()), 'value': 66.18}, {'start': datetime.datetime(2025, 1, 10, 4, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 5, 0, tzinfo=tzutc()), 'value': 75.09}, {'start': datetime.datetime(2025, 1, 10, 5, 0, tzinfo=tzutc()), 'end': datetime.datetime(2025, 1, 10, 6, 0, tzinfo=tzutc(

,start,end,value
0,2025-01-09 23:00:00+00:00,2025-01-10 00:00:00+00:00,79.76
1,2025-01-10 00:00:00+00:00,2025-01-10 01:00:00+00:00,79.74
2,2025-01-10 01:00:00+00:00,2025-01-10 02:00:00+00:00,54.95
3,2025-01-10 02:00:00+00:00,2025-01-10 03:00:00+00:00,50.94
4,2025-01-10 03:00:00+00:00,2025-01-10 04:00:00+00:00,66.18
5,2025-01-10 04:00:00+00:00,2025-01-10 05:00:00+00:00,75.09
6,2025-01-10 05:00:00+00:00,2025-01-10 06:00:00+00:00,101.21
7,2025-01-10 06:00:00+00:00,2025-01-10 07:00:00+00:00,125.63
8,2025-01-10 07:00:00+00:00,2025-01-10 08:00:00+00:00,141.30
9,2025-01-10 08:00:00+00:00,2025-01-10 09:00:00+00:00,137.30


In [ ]:
df = pd.DataFrame(estonia_prices["values"])

## Add one hour to the time
df["start"] = pd.to_datetime(df["start"]) + pd.Timedelta(hours=1)
df["timezone"] = "UTC+1"

df

,start,end,value,timezone
0,2025-01-10 00:00:00+00:00,2025-01-10 00:00:00+00:00,79.76,UTC+1
1,2025-01-10 01:00:00+00:00,2025-01-10 01:00:00+00:00,79.74,UTC+1
2,2025-01-10 02:00:00+00:00,2025-01-10 02:00:00+00:00,54.95,UTC+1
3,2025-01-10 03:00:00+00:00,2025-01-10 03:00:00+00:00,50.94,UTC+1
4,2025-01-10 04:00:00+00:00,2025-01-10 04:00:00+00:00,66.18,UTC+1
5,2025-01-10 05:00:00+00:00,2025-01-10 05:00:00+00:00,75.09,UTC+1
6,2025-01-10 06:00:00+00:00,2025-01-10 06:00:00+00:00,101.21,UTC+1
7,2025-01-10 07:00:00+00:00,2025-01-10 07:00:00+00:00,125.63,UTC+1
8,2025-01-10 08:00:00+00:00,2025-01-10 08:00:00+00:00,141.30,UTC+1
9,2025-01-10 09:00:00+00:00,2025-01-10 09:00:00+00:00,137.30,UTC+1


### Get weather data

In [16]:
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


ModuleNotFoundError: No module named 'retry_requests'

In [17]:
import requests
import pandas as pd

# Coordinates for Tallinn, Estonia
latitude = 59.437
longitude = 24.7535

url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 59.4165,
	"longitude": 24.7994,
	"hourly": ["temperature_2m", "wind_speed_10m", "direct_radiation", "diffuse_radiation"],
	"forecast_days": 1
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_direct_radiation = hourly.Variables(2).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["direct_radiation"] = hourly_direct_radiation
hourly_data["diffuse_radiation"] = hourly_diffuse_radiation

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

NameError: name 'openmeteo' is not defined